In [1]:
import os
import sys
import gzip

import pandas as pd
import numpy as np

from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder

import tqdm
from joblib import Parallel, delayed
from multiprocessing import Manager

sys.path.append('../..')
from slp.slp_package.slp_functions import create_merged_game_data_df
from slp.slp_package.input_dataset import InputDataSet

In [2]:
source_data = ['ranked']

general_features = {
    'stage_name': ['FOUNTAIN_OF_DREAMS','FINAL_DESTINATION','BATTLEFIELD','YOSHIS_STORY','POKEMON_STADIUM'],
    'num_players': [2],
    'conclusive': [True]
}
player_features = {
    # 'netplay_code': ['MANG#0'],
    'character_name': ['FOX', 'FALCO', 'MARTH', 'CAPTAIN_FALCON', 'SHEIK'],
    'type_name': ['HUMAN']
    
}
opposing_player_features = {
    # 'character_name': ['MARTH'],
    # 'netplay_code': ['KOD#0', 'ZAIN#0']
    'type_name': ['HUMAN']
}
label_info = {
    'source': ['player'], # Can be 'general', 'player
    # 'feature': ['netplay_code']
    'feature': ['character_name']
}
    

In [3]:
dataset = InputDataSet(source_data, general_features, player_features, opposing_player_features, label_info)

In [4]:
dataset.dataset.value_counts()
dataset.dataset.head()

,stage_name,num_players,conclusive,player_character_name,player_type_name,opposing_player_type_name,player_inputs_np_save_path,length,labels
0,FINAL_DESTINATION,2,True,FOX,HUMAN,HUMAN,C:\Users\jaspa\Grant ML\input_np\ranked\FOX\ae...,5969,FOX
1,BATTLEFIELD,2,True,CAPTAIN_FALCON,HUMAN,HUMAN,C:\Users\jaspa\Grant ML\input_np\ranked\CAPTAI...,6073,CAPTAIN_FALCON
2,FINAL_DESTINATION,2,True,CAPTAIN_FALCON,HUMAN,HUMAN,C:\Users\jaspa\Grant ML\input_np\ranked\CAPTAI...,6989,CAPTAIN_FALCON
3,BATTLEFIELD,2,True,SHEIK,HUMAN,HUMAN,C:\Users\jaspa\Grant ML\input_np\ranked\SHEIK\...,6806,SHEIK
4,FOUNTAIN_OF_DREAMS,2,True,FOX,HUMAN,HUMAN,C:\Users\jaspa\Grant ML\input_np\ranked\FOX\21...,7102,FOX


In [5]:
dataset.number_of_segments_per_game(10, 10000)

,Label,Count,Shift
0,FOX,47517,41528
1,FALCO,42080,37021
2,MARTH,24402,23059
3,SHEIK,15840,15878
4,CAPTAIN_FALCON,15475,13157


In [ ]:
X_train, X_test, y_train, y_test  = dataset.train_test_split_numpy(test_ratio = .30,  val = False)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


In [ ]:
from sktime.classification.deep_learning.cnn import CNNClassifier

batch_size = 32
epochs = 50
model = CNNClassifier(n_epochs=epochs, batch_size=batch_size, loss = 'categorical_crossentropy')

model.fit(X_train, y_train)